In [10]:
from acevedo_clss_and_fcns import * 
from    cobra.io.mat import *
from networkx.algorithms import bipartite
model = load_matlab_model("./COBRA_models/GEM_Recon3_thermocurated_redHUMAN_AA.mat")
rxn_list_recon: list[str] = [model.reactions[i].id       for i in range(model.reactions.__len__())]
met_list_recon: list[str] = [model.metabolites[i].id     for i in range(model.metabolites.__len__())]

G = nx.read_gpickle("./results/graphs/NX_recon_graph.gpickle")
assert nx.is_connected(G)
assert 1 == np.unique(list(nx.get_edge_attributes(G, "weight").values())).__len__()
n_nodes                            = len(list(G.nodes))
first_partition , second_partition = bipartite.sets(G)

if first_partition.__len__() > second_partition.__len__():
    rxn_partition = first_partition
    met_partition = second_partition
else:
    rxn_partition = second_partition 
    met_partition = first_partition
    
assert set(rxn_partition).issubset(set(rxn_list_recon)) and set(met_partition).issubset(set(met_list_recon))
assert len(set(rxn_partition) - set(rxn_list_recon)) == 0
assert len(set(met_partition) - set(met_list_recon)) == 0

No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, g, i, l, m, n, r, x


In [21]:
G.nodes['dcholcoa_c'], G.nodes['r2230'], G.nodes["crvnc_c"], G.nodes['r0557']

({'bipartite': 0}, {'bipartite': 0}, {'bipartite': 0}, {'bipartite': 0})

In [22]:
"dcholcoa_c" in rxn_partition, "r2230" in rxn_partition,"crvnc_c" in rxn_partition, "r0557" in rxn_partition


(False, True, False, True)

In [29]:
feature_dict = dict(zip(met_partition, itertools.repeat(0)))
feature_dict.update(dict(zip(rxn_partition, itertools.repeat(1))))



features_to_set =  {key: {"bipartite": feature_dict[key]} for key in feature_dict}
nx.set_node_attributes(G, features_to_set)

G.nodes['dcholcoa_c'], G.nodes['r2230'], G.nodes["crvnc_c"], G.nodes['r0557']

({'bipartite': 0}, {'bipartite': 1}, {'bipartite': 0}, {'bipartite': 1})

In [30]:


nx.write_gexf(G, "./results/graphs/for_visualizations/recon3_pku.gexf")


In [52]:
partition_list =  np.array(list(nx.get_node_attributes(G, "bipartite").values()))
mask_rxns      =  partition_list.astype(bool)
mask_mets      =  np.invert(partition_list.astype(bool))

In [55]:

loader_path = './results/dataloaders/MASKED_loader_Concen_plus_Fluxes.pt'
loader = torch.load(loader_path)
a_batch         = next(iter(loader.get_train_loader()))
a_graph         = a_batch[0]
assert a_graph.x.shape[0] ==len(mask_mets) ==len(partition_list)

In [81]:
from itertools import compress

node_names = list(G.nodes())

#check masks and node names
assert set(list(compress(node_names, mask_rxns)))==set(rxn_partition)
assert set(list(compress(node_names, mask_mets)))==set(met_partition)
assert a_graph.x.shape[0] ==len(mask_mets) == len(partition_list) == len(node_names)

  
feature_vector = a_graph.x.reshape(n_nodes).abs().tolist()

def get_partition_feature(feature_vector, mask, cutoff:float=1e-5):      

    metabolite_feature = list(compress(feature_vector,  mask))
    return list(compress(metabolite_feature, np.array(metabolite_feature) > cutoff))




get_partition_feature(feature_vector, mask_mets)



[15.291659355163574,
 18.827577590942383,
 43.67924118041992,
 5.411013126373291,
 0.4177795648574829,
 46.886253356933594,
 8.264768600463867,
 21.6322021484375,
 8.687117576599121,
 98.46745300292969,
 2.004115581512451,
 0.13696499168872833,
 6.187885761260986,
 0.5509090423583984,
 42.84254455566406,
 0.014073937200009823,
 0.7990325093269348,
 56.52033996582031,
 3.0313470363616943,
 4.9837646484375,
 26.809738159179688,
 1.1836073398590088,
 1.4123356342315674,
 84.92194366455078,
 17.10689926147461,
 26.25542449951172,
 0.7097868323326111,
 0.2527523338794708,
 0.4072314500808716,
 65.56473541259766,
 1.7565703392028809,
 26.385095596313477,
 90.01374816894531,
 0.20248927175998688,
 1.7073472738265991,
 0.12824195623397827,
 1.8626776933670044,
 33.58403778076172,
 25.534494400024414,
 62.8238639831543,
 1.1693940162658691,
 0.02577720582485199,
 0.2812042236328125,
 0.15490761399269104,
 10.305336952209473,
 0.12999999523162842,
 0.15951083600521088,
 15.459077835083008,
 28.5

In [ ]:

feature_dict   = dict(zip(G, feature_vector))
nx.set_node_attributes(G, feature_dict, "size")




In [ ]:
nx.write_gexf(G, "./results/graphs/for_visualizations/recon3_pku.gexf")